# **Baseline Ngẫu nhiên phân tầng (Stratified Random Baseline)**

### **Mục tiêu**

- Tính toán điểm ảnh hưởng (help/harm score) cho tập hợp 100 mẫu ngẫu nhiên đối với mỗi mẫu Test.
- Mục đích là kiểm chứng độ tin cậy của phương pháp và trả lời câu hỏi:
"Liệu các mẫu Top-k tìm được có thực sự ảnh hưởng mạnh hơn một nhóm ngẫu nhiên hay không?"

- Về cách tính, ta sẽ thực hiện giống với file  `calculate_help_harm_with_cosine_similarity.ipynb`
- Để đảm bảo tính công bằng của phép so sánh, ta áp dụng chiến lược Stratified Sampling thay vì ngẫu nhiên hoàn toàn.

### **Stratified Sampling**

- Các bước lấy 100 mẫu bao gồm:
  - Xác định nhãn của mẫu Test: VD: ARG-0
  - Lấy trong top 500 mẫu train (với mỗi mẫu test) - lấy trong thư mục `search_results/layer_wise_results_for_500_samples`, những mẫu có cùng nhãn với mẫu test.
  - Sau đó, từ danh sách những mẫu train có cùng nhãn với mẫu test, chọn ngẫu nhiên 100 mẫu.

- Ý nghĩa: Việc này giúp loại bỏ nhiễu do loại thực thể - tức là những biến động điểm số gây ra vì sự khác biệt các lớp nhãn khác nhau trong không gian biểu diễn. Vì vậy mà phép so sánh được công bằng hơn.

### **Tối ưu hóa hiệu năng**

- Vì phải tính toán cho 100 mẫu baseline cho mỗi mẫu test, việc chạy này rất chậm. Chính vì thế, ta sử dụng cơ chế Batch Processing.
- Với batch size = 32, ta gom 32 vectors train thành 1 tensor lớn (Batch, Hidden_Dim) và thực hiện phép Neutralize cùng lúc.


### **Quy trình xử lý**

- Load model và index của các file Span Adaptation Vector.
- Cấu hình để chạy một phân đoạn dữ liệu cụ thể (ví dụ từ mẫu test 0 - mẫu test 500). Mỗi lần chạy tầm 500 mẫu test. Điều này giúp chia nhỏ công việc và chạy nhanh hơn.


- Với mỗi câu test:
  1. Truy vấn Text gốc và Label gốc thông qua class TextLookup.
  2. Lọc ra danh sách các mẫu cùng nhãn,
  3. Nếu không đủ 100 ứng viên train cho 1 câu test => lấy toàn bộ. Nếu hơn 100 ứng viên, lấy ngẫu nhiên 100 mẫu train.

- Chia 100 mẫu random thành các batch nhỏ. Đưa vào Neutralizer để tính $p_{cf}$. Sau đó tính help_harm_score.
- Kết quả được lưu vào key `random_baseline_neighbors` và lưu vào thư mục `.../layer_results_with_help_harm_scores_stratified_from_top500/` với các thư mục như `1_0_to_500`

In [ ]:
import torch
import os
import json
import glob
import re
import random
import numpy as np
from tqdm.auto import tqdm
from transformers import AutoTokenizer, AutoModelForTokenClassification
import torch.nn.functional as F
from functools import lru_cache

# CẤU HÌNH
drive_base_path = '/content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep'
model_path = os.path.join(drive_base_path, 'Finetuned_Models/biobert-srl-best-model')

input_results_dir = os.path.join(drive_base_path, 'search_results/layer_wise_results_for_500_samples')
output_results_dir = os.path.join(drive_base_path, 'search_results/v2_neutralize_with_cosine_similarity/layer_results_with_help_harm_scores_stratified_from_top500/4_1500_to_2000')
os.makedirs(output_results_dir, exist_ok=True)

BATCH_SIZE = 32

VECTOR_DIRS = [
    os.path.join(drive_base_path, 'Span Adaptation Vector/With Weight/Train/span_adaptation_vectors_train_gramvar_inner_content'),
    os.path.join(drive_base_path, 'Span Adaptation Vector/With Weight/Train/span_adaptation_vectors_train_parave_inner_content'),
    os.path.join(drive_base_path, 'Span Adaptation Vector/With Weight/Test/span_adaptation_vectors_test_gramvar_inner_content'),
    os.path.join(drive_base_path, 'Span Adaptation Vector/With Weight/Test/span_adaptation_vectors_test_parave_inner_content'),
]

DATASET_ROOT = os.path.join(drive_base_path, 'Clean_Dataset/Corpus')
DATASET_DIRS = [
    os.path.join(DATASET_ROOT, 'Split_GramVar/Train'),
    os.path.join(DATASET_ROOT, 'Split_GramVar/Test'),
    os.path.join(DATASET_ROOT, 'Split_ParaVE/Train'),
    os.path.join(DATASET_ROOT, 'Split_ParaVE/Test'),
]

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")
if torch.cuda.is_available():
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")

# Cache và lookup
file_path_map = {}
dataset_json_cache = {}

def build_file_map():
    print("[INIT] Đang quét file vector...")
    count = 0
    for root_dir in VECTOR_DIRS:
        if not os.path.exists(root_dir): continue
        for root, dirs, files in os.walk(root_dir):
            corpus_id = os.path.basename(root)
            for f in files:
                if f.endswith('.pt'):
                    file_path_map[(corpus_id, f)] = os.path.join(root, f)
                    count += 1
    print(f"[INIT] Đã index {count} vector files.")

@lru_cache(maxsize=10000)
def load_torch_file_cached(path):
    try: return torch.load(path, map_location='cpu')
    except: return None

def get_file_path_robust(corpus_id, sent_idx, span_id, absolute_path_hint=None):
    fname1 = f"sentence_{sent_idx}_{span_id}.pt"
    if (corpus_id, fname1) in file_path_map: return file_path_map[(corpus_id, fname1)]

    fname2 = f"sentence_{sent_idx}_{str(span_id).replace('-', '_')}.pt"
    if (corpus_id, fname2) in file_path_map: return file_path_map[(corpus_id, fname2)]

    return absolute_path_hint if absolute_path_hint and os.path.exists(absolute_path_hint) else None

def load_pt_data(corpus_id, sent_idx, span_id, layer_idx, absolute_path=None):
    path = get_file_path_robust(corpus_id, sent_idx, span_id, absolute_path)
    if not path: return None, None
    data = load_torch_file_cached(path)
    if data is None: return None, None

    vec, weights = None, None
    layer_key, weight_key = f"layer_{layer_idx}", f"layer_{layer_idx}_token_weights"

    if isinstance(data, dict):
        vec, weights = data.get(layer_key), data.get(weight_key)
        if vec is None:
            for k, v in data.items():
                if torch.is_tensor(v) and "token_weights" not in k: vec = v; break
    elif torch.is_tensor(data):
        vec, weights = data[:-13], data[-13:]
    return vec, weights

class TextLookup:
    def __init__(self):
      self.path_cache = {}
      for d in DATASET_DIRS:
          if not os.path.exists(d): print(f"⚠️ Missing: {d}")

    def get_text_and_label(self, corpus_id, sent_idx, span_id):
        json_path = self.path_cache.get(corpus_id)
        if not json_path:
            for d in DATASET_DIRS:
                for name in [f"{corpus_id}_test_set.json", f"{corpus_id}_train_set.json"]:
                    p = os.path.join(d, name)
                    if os.path.exists(p): json_path = p; break
                if json_path: break
            if json_path: self.path_cache[corpus_id] = json_path
            else: return None, None, None

        if json_path not in dataset_json_cache:
            with open(json_path, 'r', encoding='utf-8') as f:
              dataset_json_cache[json_path] = json.load(f)

        data = dataset_json_cache[json_path]
        if sent_idx >= len(data): return None, None, None

        entry = data[sent_idx]
        text = entry.get('text', '')
        args = entry.get('arguments', {})
        search_key = str(span_id).replace('_', '-')
        arg_info = args.get(search_key)

        if isinstance(arg_info, dict):
             return text, arg_info.get('text', ''), arg_info.get('label', '')
        elif isinstance(arg_info, str):
             return text, arg_info, ''
        return text, '', ''

# Neutralize
class Neutralizer:
    def __init__(self, model):
        self.model = model
        self.device = next(model.parameters()).device
        self.hidden_size = 768

    def get_counterfactual_prob_batched(self, inputs, layer_idx, train_vecs_list, test_vec_full, dists_list, span_mask, test_token_weights, target_label_idx):
        batch_size = len(train_vecs_list)
        if batch_size == 0: return [], []

        w_cosines = []
        final_weights = []

        test_vec_gpu = test_vec_full.to(self.device) if test_vec_full is not None else None

        for i in range(batch_size):
            t_vec = train_vecs_list[i].to(self.device)
            dist = dists_list[i]

            w_mahalanobis = 1.0 / (1.0 + dist)
            w_cosine = 0.0
            if test_vec_gpu is not None:
                cos_sim = F.cosine_similarity(test_vec_gpu, t_vec, dim=0).item()
                w_cosine = max(0.0, cos_sim)
            else:
                w_cosine = 1.0

            w_cosines.append(w_cosine)
            final_weights.append(w_mahalanobis * w_cosine)

        batch_train_vecs = torch.stack(train_vecs_list).to(self.device)
        batch_weights = torch.tensor(final_weights, device=self.device).view(-1, 1)

        vec_starts = batch_train_vecs[:, 0 : self.hidden_size]
        vec_ends   = batch_train_vecs[:, self.hidden_size : 2*self.hidden_size]
        vec_contents = batch_train_vecs[:, 2*self.hidden_size :]

        sub_starts = vec_starts * batch_weights
        sub_ends   = vec_ends * batch_weights
        sub_contents = vec_contents * batch_weights

        batched_input_ids = inputs['input_ids'].expand(batch_size, -1)
        batched_attention_mask = inputs['attention_mask'].expand(batch_size, -1)
        batched_inputs = {
            'input_ids': batched_input_ids,
            'attention_mask': batched_attention_mask
        }
        if 'token_type_ids' in inputs:
            batched_inputs['token_type_ids'] = inputs['token_type_ids'].expand(batch_size, -1)

        def batch_neutralization_hook(module, input, output):
            is_tuple = isinstance(output, tuple)
            hidden_states = output[0] if is_tuple else output

            indices = torch.where(span_mask[0] == 1)[0]
            if len(indices) == 0: return output

            hidden_states[:, indices[0], :] -= sub_starts

            if len(indices) > 1:
                hidden_states[:, indices[-1], :] -= sub_ends

            if len(indices) > 2:
                inner_indices = indices[1:-1]
                if test_token_weights is not None and len(test_token_weights) == len(indices):
                    inner_weights = test_token_weights[1:-1].to(self.device)
                    to_subtract = inner_weights.view(1, -1, 1) * sub_contents.view(batch_size, 1, -1)
                    hidden_states[:, inner_indices, :] -= to_subtract

            if is_tuple: return (hidden_states,) + output[1:]
            return hidden_states

        layer_module = self.model.bert.encoder.layer[layer_idx]
        hook_handle = layer_module.register_forward_hook(batch_neutralization_hook)

        try:
            with torch.no_grad():
                out_cf = self.model(**batched_inputs)
                probs_cf = F.softmax(out_cf.logits, dim=-1)

            span_idxs = torch.where(span_mask[0] == 1)[0]
            p_cfs = probs_cf[:, span_idxs, target_label_idx].mean(dim=1).cpu().tolist()

        finally:
            hook_handle.remove()

        return p_cfs, w_cosines

# Các hàm hỗ trợ
def load_input_text_data(sentence_text, argument_text, corpus_id, span_id, tokenizer, label2id):
    if not sentence_text or not argument_text: return None, None, None
    inputs = tokenizer(sentence_text, return_tensors="pt", truncation=True, max_length=512).to(device)
    arg_ids = tokenizer.encode(argument_text.strip(), add_special_tokens=False)
    input_ids = inputs.input_ids[0].cpu().numpy()
    n, m = len(input_ids), len(arg_ids)
    start_idx = -1
    for i in range(n - m + 1):
        if np.array_equal(input_ids[i:i+m], arg_ids):
            start_idx = i; break
    if start_idx == -1: return None, None, None

    span_mask = torch.zeros_like(inputs.input_ids)
    span_mask[0, start_idx : start_idx + m] = 1

    clean_span = str(span_id).replace('_', '-')
    target_id = None
    for k, v in label2id.items():
         if clean_span in k and k.startswith("B-"):
             target_id = v; break

    return inputs, span_mask, target_id

def get_query_ids(query):
    c, s, idx = query.get('corpus_id'), query.get('span_id'), query.get('sentence_idx')
    if not c or not s or idx is None:
        meta = query.get('metadata', {})
        c, s, idx = meta.get('corpus_id', c), meta.get('span_id', s), meta.get('sentence_idx', idx)
    return c, s, idx

# Hàm main
def main():
    build_file_map()
    print("[1] Loading Model...")
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForTokenClassification.from_pretrained(model_path).to(device)
    model.eval()
    neutralizer, lookup = Neutralizer(model), TextLookup()
    label2id = model.config.label2id

    json_files = sorted(glob.glob(os.path.join(input_results_dir, "search_results_layer_*.json")))
    def get_lnum(f):
      m = re.search(r'layer_(\d+)', f)
      return int(m.group(1)) if m else 999
    json_files.sort(key=get_lnum)
    print(f"[2] Tìm thấy {len(json_files)} file kết quả. Bắt đầu xử lý với Batch Size {BATCH_SIZE}...")

    for json_file in tqdm(json_files, desc="Processing Layers"):
        layer_num = get_lnum(os.path.basename(json_file))
        layer_idx = layer_num - 1

        with open(json_file, 'r', encoding='utf-8') as f:
          data = json.load(f)

        out_name = os.path.basename(json_file)
        final_path = os.path.join(output_results_dir, out_name)
        updated_data = []

        # Slice 1500:2000
        current_batch_data = data[1500:2000]
        if not current_batch_data: continue

        for entry in tqdm(current_batch_data, desc=f"L{layer_num}", leave=False):
            if 'neighbors' not in entry or len(entry['neighbors']) <= 5: continue

            q = entry.get('query_info', {})
            cid, sid, sidx = get_query_ids(q)

            s_txt, a_txt, test_label = lookup.get_text_and_label(cid, sidx, sid)
            if not s_txt or not a_txt: continue

            inputs, span_mask, target_id = load_input_text_data(
                s_txt, a_txt, cid, sid, tokenizer, label2id
            )
            if inputs is None or target_id is None: continue

            with torch.no_grad():
                out_orig = model(**inputs)
                probs_orig = F.softmax(out_orig.logits, dim=-1)[0]
                span_idxs = torch.where(span_mask[0] == 1)[0]
                if len(span_idxs) == 0: continue
                p_orig = probs_orig[span_idxs, target_id].mean().item()

            test_vec, t_weights = load_pt_data(cid, sidx, sid, layer_idx)
            if t_weights is not None:
                t_weights = t_weights.float()
                if t_weights.sum() > 0: t_weights /= t_weights.sum()

            # Lấy những giá trị nào có mẫu giống với mẫu test
            candidates = []
            potential_pool = entry['neighbors'][5:]
            for nb in potential_pool:
                m_meta = nb.get('match_metadata', {})
                _, _, nb_label = lookup.get_text_and_label(
                    m_meta.get('corpus_id'), m_meta.get('sentence_idx'), m_meta.get('span_id')
                )

                if nb_label == test_label:
                    candidates.append(nb)

            if len(candidates) > 100:
                random_samples = random.sample(candidates, 100)
            else:
                random_samples = candidates

            if not random_samples: continue

            # xử lý theo lô size = 32
            processed_neighbors = []

            for i in range(0, len(random_samples), BATCH_SIZE):
                batch_nbs = random_samples[i : i + BATCH_SIZE]

                batch_train_vecs = []
                batch_dists = []
                valid_indices = []

                for idx_in_batch, nb in enumerate(batch_nbs):
                    m_meta = nb.get('match_metadata', {})
                    train_vec, _ = load_pt_data(
                        m_meta.get('corpus_id'),
                        m_meta.get('sentence_idx'),
                        m_meta.get('span_id'),
                        layer_idx,
                        m_meta.get('vector_file_path')
                    )

                    if train_vec is not None:
                        batch_train_vecs.append(train_vec)
                        batch_dists.append(nb.get('l2_distance', 0.0))
                        valid_indices.append(idx_in_batch)

                if batch_train_vecs:
                    batch_p_cfs, batch_w_cos = neutralizer.get_counterfactual_prob_batched(
                        inputs=inputs,
                        layer_idx=layer_idx,
                        train_vecs_list=batch_train_vecs,
                        test_vec_full=test_vec,
                        dists_list=batch_dists,
                        span_mask=span_mask,
                        test_token_weights=t_weights,
                        target_label_idx=target_id
                    )

                    for k, valid_idx in enumerate(valid_indices):
                        nb_obj = batch_nbs[valid_idx]
                        p_cf_val = batch_p_cfs[k]
                        w_cos_val = batch_w_cos[k]

                        nb_obj.update({
                            'help_harm_score': p_orig - p_cf_val,
                            'p_orig': p_orig,
                            'p_cf': p_cf_val,
                            'cosine_weight': w_cos_val,
                            'is_stratified_filtered': True
                        })
                        processed_neighbors.append(nb_obj)

            if processed_neighbors:
                # Gán kết quả tính toán (Chỉ chứa các neighbors cùng nhãn)
                entry['random_baseline_neighbors'] = processed_neighbors

                if 'neighbors' in entry:
                    del entry['neighbors']

                updated_data.append(entry)

                if len(updated_data) % 10 == 0:
                    with open(final_path, 'w', encoding='utf-8') as f: json.dump(updated_data, f, indent=2)

        with open(final_path, 'w', encoding='utf-8') as f: json.dump(updated_data, f, indent=2)

    print(f"\nThực hiện xong Batch 1000-1500")

if __name__ == "__main__":
    main()

Device: cuda
GPU Name: Tesla T4
[INIT] Đang quét file vector...
[INIT] Đã index 17930 vector files.
[1] Loading Model...
[2] Tìm thấy 12 file kết quả. Bắt đầu xử lý với Batch Size 32...


Processing Layers:   0%|          | 0/12 [00:00<?, ?it/s]

L1:   0%|          | 0/500 [00:00<?, ?it/s]

L2:   0%|          | 0/500 [00:00<?, ?it/s]

L3:   0%|          | 0/500 [00:00<?, ?it/s]

...

L9:   0%|          | 0/500 [00:00<?, ?it/s]

L10:   0%|          | 0/500 [00:00<?, ?it/s]

L11:   0%|          | 0/500 [00:00<?, ?it/s]

L12:   0%|          | 0/500 [00:00<?, ?it/s]


Thực hiện xong Batch 1000-1500
